In [165]:
#Chiefs Data Scrape using http://pro-football-reference.com


import pandas as pd
from bs4 import BeautifulSoup as bs
import sys 
import os
from urllib.request import urlopen
import sklearn
import numpy as np
import re

import time


#change max_columns in pandas

pd.set_option('max_columns', 50)

#start scipt clock
time1 = time.time()

print(sys.version)

3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]


In [166]:
#determine how many years of data you would like to scrape

years = list(range(2005,2019))

years[-1]

2018

In [167]:
print(sys.version)

3.6.7 (default, Oct 22 2018, 11:32:17) 
[GCC 8.2.0]


In [0]:
#single year pull
url = 'https://www.pro-football-reference.com/teams/kan/2017.htm'

In [0]:
page = urlopen(url)
soup = bs(page)
table = soup.find_all("table")
table = table[1]

In [0]:
#pull for all years
for year in years:
    if year == years[0]:
        predf = dict()
        week = 0
        urls = 'https://www.pro-football-reference.com/teams/kan/'
        end = '.htm'
        url = urls+str(year)+end
        page = urlopen(url).read()
        soup = bs(page)
        table = soup.find_all("table")
        table = table[1]
        rows = table.find_all('tr')
        for row in rows:
            a = row.find_all('td')
            for rowrow in a:
                b = rowrow.text.strip().encode()
                c = b.decode('utf-8')
                ff = rowrow['data-stat']
                if ff in predf:
                    predf[ff].append(c)
                else:
                    predf[ff] = [c]
        df1 = pd.DataFrame.from_dict(predf)
        aa = []
        for index,row in df1.iterrows():
            aa.append(index + 1)
        df1['week'] = aa
        df1['season'] = year
    else:
        predf = dict()
        urls = 'https://www.pro-football-reference.com/teams/kan/'
        end = '.htm'
        url = urls+str(year)+end
        page = urlopen(url).read()
        soup = bs(page)
        table = soup.find_all("table")
        table = table[1]
        rows = table.find_all('tr')
        for row in rows:
            a = row.find_all('td')
            for rowrow in a:
                b = rowrow.text.strip().encode()
                c = b.decode('utf-8')
                ff = rowrow['data-stat']
                if ff in predf:
                    predf[ff].append(c)
                else:
                    predf[ff] = [c]
        df2 = pd.DataFrame.from_dict(predf)
        aa = []
        for index,row in df2.iterrows():
            aa.append(index + 1)
        df2['week'] = aa
        df2['season'] = year
        df1 = pd.concat([df1,df2],axis = 0,ignore_index=True)
            

In [0]:
#rearrange columns so season and week are first
df1 = pd.concat([df1[['season','week']],df1.iloc[:,:-2]],axis = 1)

In [172]:
df1.iloc[176:178,:]

,season,week,game_day_of_week,game_date,game_time,boxscore_word,game_outcome,overtime,team_record,game_location,opp,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,exp_pts_off,exp_pts_def,exp_pts_st
176,2015,1,Sun,September 13,1:02PM ET,boxscore,W,,1-0,@,Houston Texans,27,20,19,330,233,97,,24,396,298,98,2,1.05,4.01,2.24
177,2015,2,Thu,September 17,8:29PM ET,boxscore,L,,1-1,,Denver Broncos,24,31,20,314,167,147,5,21,299,238,61,1,-9.40,6.63,-5.00


In [0]:
#add feature for playoff games
df1['playoff_game'] = 'no'
aa = []
for index, row in df1.iterrows():
    if row['game_location'] == '@':
        aa.append('away')
    else:
        aa.append('home')
df1['game_location'] = aa

In [0]:
for index, row in df1.iterrows():
    if row['week'] > 17:
        df1.loc[index,'week'] = df1.loc[index,'week'] - 1
        df1.loc[index,'playoff_game'] = 'yes'
        

In [0]:
df1 = df1[df1['game_date'] != 'Playoffs']
del[df1['boxscore_word']]

In [176]:
print('total script time', round((time.time() - time1) / 60,2),'minutes')

total script time 0.18 minutes


In [0]:
#get attendance html

url = 'https://www.pro-football-reference.com/years/2006/attendance.htm'
page = urlopen(url).read()

soup = bs(page)

In [0]:
table = soup.find("table", {'id' : 'attendance'})

In [179]:
predf = dict()
rows = table.find_all('tr')


for row in rows:
  b = row.find_all('th')[0].text.strip()
  if b == 'Tm':
    pass
  elif 'team' not in predf:
    print(b)
    predf['team'] = [b]
  else:
    predf['team'].append(b)

for row in rows:
  a = row.find_all('td')
  for rowrow in a:
      b = rowrow.text.strip().encode()
      c = b.decode('utf-8')
      ff = rowrow['data-stat']
      if ff in predf:
          predf[ff].append(c)
      else:
          predf[ff] = [c]

Arizona Cardinals


In [180]:
a = pd.DataFrame(predf)

aa = ['team']

for n in list(range(1,18)):
  aa.append('week'+str(n))
  
b = a[a['team'] == 'Kansas City Chiefs'][aa].transpose().reset_index()
 
b.columns = ['week','attendance']


b = b.iloc[1:,:].reset_index()

del[b['index']]

b.head()

,week,attendance
0,week1,"77,956"
1,week2,"76,786"
2,week3,Bye
3,week4,"77,609"
4,week5,"63,445"


In [0]:
#make dataset of all years for attendance

for year in years:
  if year == years[0]:
    url = 'https://www.pro-football-reference.com/years/' + str(year) + '/attendance.htm'
    page = urlopen(url).read()
    soup = bs(page)
    table = soup.find("table", {'id' : 'attendance'})
    predf = dict()
    rows = table.find_all('tr')

    for row in rows:
      b = row.find_all('th')[0].text.strip()
      if b == 'Tm':
        pass
      elif 'team' not in predf:
        #print(b)
        predf['team'] = [b]
      else:
        predf['team'].append(b)

    for row in rows:
      a = row.find_all('td')
      for rowrow in a:
          b = rowrow.text.strip().encode()
          c = b.decode('utf-8')
          ff = rowrow['data-stat']
          if ff in predf:
              predf[ff].append(c)
          else:
              predf[ff] = [c]
    a = pd.DataFrame(predf)

    aa = ['team']

    for n in list(range(1,18)):
      aa.append('week'+str(n))

    attend = a[a['team'] == 'Kansas City Chiefs'][aa].transpose().reset_index()

    attend.columns = ['week','attendance']


    attend = attend.iloc[1:,:].reset_index()

    del[attend['index']]
    attend['year'] = year
  else:
    url = 'https://www.pro-football-reference.com/years/' + str(year) + '/attendance.htm'
    page = urlopen(url).read()
    soup = bs(page)
    table = soup.find("table", {'id' : 'attendance'})
    predf = dict()
    rows = table.find_all('tr')

    for row in rows:
      b = row.find_all('th')[0].text.strip()
      if b == 'Tm':
        pass
      elif 'team' not in predf:
        #print(b)
        predf['team'] = [b]
      else:
        predf['team'].append(b)

    for row in rows:
      a = row.find_all('td')
      for rowrow in a:
          b = rowrow.text.strip().encode()
          c = b.decode('utf-8')
          ff = rowrow['data-stat']
          if ff in predf:
              predf[ff].append(c)
          else:
              predf[ff] = [c]
    a = pd.DataFrame(predf)

    aa = ['team']

    for n in list(range(1,18)):
      aa.append('week'+str(n))

    attend1 = a[a['team'] == 'Kansas City Chiefs'][aa].transpose().reset_index()

    attend1.columns = ['week','attendance']


    attend1 = attend1.iloc[1:,:].reset_index()

    del[attend1['index']]
    attend1['year'] = year
    attend = pd.concat([attend,attend1], axis = 0)
    




In [182]:
attend.head()

,week,attendance,year
0,week1,"78,014",2005
1,week2,"62,273",2005
2,week3,"76,381",2005
3,week4,"78,742",2005
4,week5,Bye,2005


In [0]:
#make unique key out of year and week

aa = []

for index,row in attend.iterrows():
  year = str(row['year'])
  week = re.sub('[a-z]', '', row['week'])
  year_week = year + week
  aa.append(year_week)
  
attend['year_week'] = aa

In [184]:
attend.head()

,week,attendance,year,year_week
0,week1,"78,014",2005,20051
1,week2,"62,273",2005,20052
2,week3,"76,381",2005,20053
3,week4,"78,742",2005,20054
4,week5,Bye,2005,20055


In [185]:
df1.head()

,season,week,game_day_of_week,game_date,game_time,game_outcome,overtime,team_record,game_location,opp,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,exp_pts_off,exp_pts_def,exp_pts_st,playoff_game
0,2005,1,Sun,September 11,1:00PM ET,W,,1-0,home,New York Jets,27,7,23,389,191,198,1,19,390,333,57,3,12.26,11.60,-5.00,no
1,2005,2,Sun,September 18,8:30PM ET,W,,2-0,away,Oakland Raiders,23,17,17,354,229,125,1,16,327,255,72,2,4.05,-1.52,1.52,no
2,2005,3,Mon,September 26,9:07PM ET,L,,2-1,away,Denver Broncos,10,30,18,285,211,74,1,24,358,137,221,,-1.23,-19.32,-2.42,no
3,2005,4,Sun,October 2,4:15PM ET,L,,2-2,home,Philadelphia Eagles,31,37,19,353,209,144,4,25,396,368,28,2,-3.72,-10.01,9.11,no
4,2005,5,,,,,,,home,Bye Week,,,,,,,,,,,,,,,,no


In [0]:
#create year_week for main dataframe

aa = [] 

for index,row in df1.iterrows():
  year = str(row['season'])
  week = str(row['week'])
  aa.append(year + week)
  
df1['year_week'] = aa

In [0]:
df = df1.merge(attend[['attendance','year_week']], on = 'year_week')

#delete matching key
del[df['year_week']]

In [188]:
df.tail()

,season,week,game_day_of_week,game_date,game_time,game_outcome,overtime,team_record,game_location,opp,pts_off,pts_def,first_down_off,yards_off,pass_yds_off,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,exp_pts_off,exp_pts_def,exp_pts_st,playoff_game,attendance
233,2018,13,Sun,December 2,4:05PM ET,W,,10-2,away,Oakland Raiders,40,33,25,469,295,174,1,28,442,271,171,3,20.87,-14.50,3.48,no,"54,255"
234,2018,14,Sun,December 9,1:00PM ET,W,OT,11-2,home,Baltimore Ravens,27,24,30,442,348,94,1,24,321,127,194,1,17.44,-3.26,-10.59,no,"74,336"
235,2018,15,Thu,December 13,8:20PM ET,L,,11-3,home,Los Angeles Chargers,28,29,22,294,234,60,,30,407,288,119,2,17.10,-21.67,2.83,no,"75,091"
236,2018,16,Sun,December 23,8:20PM ET,L,,11-4,away,Seattle Seahawks,31,38,22,419,265,154,2,31,464,254,210,,11.26,-23.20,5.47,no,"69,067"
237,2018,17,Sun,December 30,4:25PM ET,W,,12-4,home,Oakland Raiders,35,3,18,409,310,99,1,20,292,165,127,4,17.34,22.66,-9.26,no,"77,550"
